In [1]:
import socket
import pickle
import threading
import random
import math
import time
#быстрое возведение в степень больших чисел
def fast_pow(x, y, m):
    if y == 0:
        return 1
    if y == -1:
        return 1. / x
    p = fast_pow(x, y // 2, m)
    p *= p
    p = p % m
    if y % 2:
        p *= x
        p = p % m
    return p
#тест миллера-рабина
def is_Prime(n,k):
    if k==0:
        k=int(math.log(n))
    if n==0 or n==1 or n==4 or n==6 or n==8 or n==9:
        return False
    if n==2 or n==3 or n==5 or n==7:
        return True
    s = 0
    d = n-1
    while d%2==0:
        d>>=1
        s+=1
    assert(2**s * d == n-1)
    def trial_composite(a):
        if fast_pow(a, d, n) == 1:
            return False
        for i in range(s):
            if fast_pow(a, 2**i * d, n) == n-1:
                return False
        return True  
    for i in range(k):
        a = random.randrange(2, n-1)
        if trial_composite(a):
            return False
    return True
#функция генерации рандомных простых чисел заданной длины
def RandomPQ(length):
    while(True):
        p=random.randrange(pow(2,length)+1, pow(2,length+1))
        if(is_Prime(p,0)):
            break
    while(True):
        q=random.randrange(pow(2,length)+1, pow(2,length+1))
        if(is_Prime(q,0)):
            break
    return p,q
#функция нахождения коэф безу и нод
def bezu(a, b):
        if(a < b):
            a, b = b, a
        x0, x1, y0, y1 = 1, 0, 0, 1
        while b:
            q = a // b
            a, b = b, a % b
            x0, x1 = x1, x0 - x1*q
            y0, y1 = y1, y0 - y1*q
        return (x0, y0, a)
#РАЕ
def Evklid(a,b,m):
    if(bezu(a, m)[2] == 1):
        rez = bezu(a, m)[1]
        rez = (b * rez) % m
        return rez
    else:
        return 0
#расшифрока и зашифровка RSA
def encrypt(message,e,n):
    encrypted=fast_pow(message,e,n)
    return encrypted
def decrypt(message,d,n):
    decrypted=fast_pow(message,d,n)
    return decrypted
#генерация открытого и закрытого ключа
def generateKeys(length):
    p,q=RandomPQ(length)
    n=p*q
    s=(p-1)*(q-1)
    while(True):
        e=random.randrange(pow(2,length)+1, pow(2,length+1))
        if(bezu(e,s)[2]==1 and e<s):
            break
    d=Evklid(e,1,s)
    return d,e,n
#кодирование строки с разбиением на блоки
def encodeStr(s,length):
    s=s.lower()
    alph=' abcdefghijklmnopqrstuvwxyz'
    rez=[]
    a=0
    i=0
    st=int(math.log(len(alph),2))
    for ch in s:
        if(i*st>=length):
            rez.append(a)
            i=0
            a=0
        a+=alph.index(ch)*len(alph)**i
        i+=1
    rez.append(a)
    return rez
#раскодирование строки
def decodeStr(integer):
    alph=' abcdefghijklmnopqrstuvwxyz'
    result = ''
    integer = abs(integer)
    while integer > 0:
        integer, remainder = divmod(integer, len(alph))
        result = alph[remainder] + result

    return result[::-1]
E=0
N=0
e=0
n=0
d=0
#отправка
def send(N,E):
    message=input('Type your message: ')
    message = encodeStr(message,length)
    enc=[]
    for i in message:
        enc.append(encrypt(i,E,N))
    data={"type": "String","data": enc}
    data=pickle.dumps(data)
    sock.sendto(data, server)
#прием
def recv(n,d):
    data , address = sock.recvfrom(1024)
    data=pickle.loads(data)
    print ('Address: ',address[0], address[1],'Message: ',data)
    if data['type']=='String':
        s=''
        enc=data['data']
        for i in enc:
            s+=decodeStr(decrypt(i,d,n))
        print('MESSAGE: '+s)
        
        
server = 'localhost', 5050  # Сокс кому отправляем данные
sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
sock.bind(('localhost', 5051)) # Сокет этого приложения
rec=True
length=int(input('Введите размер генерируемых чисел(значение бит)'))
d,e,n=generateKeys(length)
while 1:
    if rec:
        while 1:
            data , address = sock.recvfrom(1024)
            data=pickle.loads(data)
            if data['type']=='Open key':
                print('key recieved')
                E=data['E']
                N=data['N']
                rec=False
                data_send={"type": "Open key","E": e,"N": n}
                data_send=pickle.dumps(data_send)
                sock.sendto(data_send, server) 
                break
    thread_send = threading.Thread(target=send,args=(N,E))
    thread_recv = threading.Thread(target=recv,args=(n,d))
    thread_send.start()
    thread_recv.start()
    thread_send.join()
    thread_recv.join()
    time.sleep(0.5)
        

Введите размер генерируемых чисел(значение бит)1024
